# 나와 닮은 연예인 찾기
## 1. 사진 모으기

## 2. 얼굴 영역 자르기
- from PIL import Image
- face = get_gropped_face(image_path)
- pillow_image = Image.fromarray(face)
- pillow_image.save(path_to_save)

In [1]:
import face_recognition
import os
from PIL import Image

dir_path = os.getenv('HOME')+'/aiffel/face_embedding/images'
file_list = os.listdir(dir_path)

print(len(file_list))

402


In [2]:
# 이미지 파일에서 얼굴 영역을 가져오는 함수
def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
    
    return cropped_face

In [3]:
# 얼굴 영역을 잘라서 이미지로 저장하는 함수
def save_cropped_face(dir_path):
    file_list = os.listdir(dir_path)
    
    for file in file_list:
        try:
            img_path = os.path.join(dir_path, file)
            face = get_gropped_face(img_path)
            pillow_image = Image.fromarray(face)
            path_to_save = os.path.join(save_path, file)
            pillow_image.save(path_to_save)
        except:
            print(file, "face not detected")
    
    print('Cropped images are saved')

save_path = os.getenv('HOME')+'/aiffel/face_embedding/images_cropped'
save_cropped_face(dir_path)

Cropped images are saved


## 3. 얼굴 영역의 임베딩 추출하기
- def get_face_embedding(face) : 얼굴영역으로부터 얼굴 임베딩 벡터를 구하는 함수
- def get_face_embedding_dict(dir_path) : 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수

In [4]:
# 얼굴 영역으로부터 얼굴 임베딩 벡터를 구하는 함수
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

In [5]:
# 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_gropped_face(img_path)      # 얼굴 영역을 구하는 함수
        embedding = get_face_embedding(face)   # 얼굴 임베딩 벡터를 구하는 함수
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면 len(embedding)==0인 경우가 발생하므로 
            # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
    
    print("Embedding dictionary was made")
    return embedding_dict

# 내 사진의 임베딩 딕셔너리 출력
embedding_dict = get_face_embedding_dict(dir_path)
embedding_dict['고은채1']

Embedding dictionary was made


array([-0.12784436,  0.10448948,  0.00994151, -0.11465763, -0.14201827,
       -0.0666955 , -0.07937627, -0.07665567,  0.14679766, -0.11085141,
        0.2065116 , -0.11440607, -0.21771631, -0.09120598, -0.08811358,
        0.25951171, -0.2395359 , -0.13590126, -0.00925686,  0.00656124,
        0.15065549,  0.01079155, -0.05236992,  0.07051725, -0.11669068,
       -0.31546485, -0.05581702, -0.06806715, -0.0489037 , -0.09101848,
       -0.0101376 ,  0.0581856 , -0.17303206, -0.02759539,  0.02669946,
        0.08334157,  0.00140176, -0.10138684,  0.15911688, -0.00163643,
       -0.28361756, -0.02073763,  0.11070047,  0.18708329,  0.17379852,
       -0.01911343,  0.0057533 , -0.15386668,  0.15663283, -0.16263032,
       -0.04790429,  0.14580764,  0.05357089,  0.07113975,  0.01887064,
       -0.12331574,  0.06889544,  0.16526967, -0.13515526, -0.03613933,
        0.10599857, -0.1019676 , -0.01170274, -0.10200847,  0.21783955,
        0.04580704, -0.11585851, -0.17724352,  0.15315324, -0.10

## 4. 모은 연예인들과 비교하기
- get_nearest_face(name, top=5)

In [6]:
import numpy as np

# 두 이미지 사이의 임베딩 벡터 거리를 구하는 함수
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

# 내 사진들끼리의 얼굴 임베딩 벡터 거리 계산
print(get_distance('고은채1', '고은채2'))
print(get_distance('고은채1', '고은채3'))
print(get_distance('고은채2', '고은채3'))

0.3553166411438206
0.26841511676106095
0.2876819917074476


In [7]:
# name1과 name2의 거리를 비교하는 함수를 생성하되, name1은 미리 지정하고, name2는 호출시에 인자로 받도록 합니다.
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

In [14]:
# 거리가 가장 가까운 순으로 출력해주는 함수
def get_nearest_face(name, top=7):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))

    for i in range(top+1):
        if i == 0 :   # 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))

In [34]:
# 거리가 가장 가까운 순으로 출력해주는 함수
def get_nearest_face(name, top=7):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))
    
    idx = 1
    for i in range(top+1):
        if i == 0 :   # 첫 번째로 나오는 이름은 자기 자신이므로 제외
            continue
        elif sorted_faces[i][0].startswith('고은채'):   # 그 외 내 사진도 제외
            continue
        else:
            print('순위 {} : 이름({}), 거리({})'.format(idx, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))
            idx += 1

In [35]:
# 나와 가장 닮은 사람은 누굴까요?
get_nearest_face('고은채1')

순위 1 : 이름(박소담3), 거리(0.33230266514098106)
순위 2 : 이름(박소담4), 거리(0.3423619454554579)
순위 3 : 이름(한채아), 거리(0.3438648248751277)
순위 4 : 이름(박소담), 거리(0.3604217294737214)
순위 5 : 이름(박보영), 거리(0.36116937133894456)


In [36]:
get_nearest_face('고은채2')

순위 1 : 이름(박보영4), 거리(0.3559747794026571)
순위 2 : 이름(박소담4), 거리(0.35937580502676747)
순위 3 : 이름(박보영3), 거리(0.36218878376892777)
순위 4 : 이름(박소담), 거리(0.36656870090720045)
순위 5 : 이름(김고은5), 거리(0.37263632218191534)


In [37]:
get_nearest_face('고은채3')

순위 1 : 이름(박소담), 거리(0.32252258812889917)
순위 2 : 이름(박소담4), 거리(0.33412241465104964)
순위 3 : 이름(박소담3), 거리(0.3474402452138631)
순위 4 : 이름(박보영4), 거리(0.3623101656441749)
순위 5 : 이름(박보영3), 거리(0.3644643216667045)


## 5. 다양한 재미있는 시각화 시도해 보기